**Unicode Strings**
Unicode is a standard is encoding system that is used to represent character from almost all languages.
Each character is encoded is using a unique integer code point between 0 to 0x10FFFF.
Unicode string is a sequence of zero or more code points.


tf.string data type allows you to build tensors of byte strings.

In [3]:
import tensorflow as tf

tf.constant(u"Thanks 😊")

<tf.Tensor: shape=(), dtype=string, numpy=b'Thanks \xf0\x9f\x98\x8a'>

A tf.string tensor can hold byte strings of varying lengths because the byte strings are treated as atomic units. The string length is not included in the tensor dimensions.


In [4]:
tf.constant([u"You're", u"welcome!"]).shape

TensorShape([2])

Represting Unicode

Two standard ways to represent a unicode string in tensorflow

string scalar -  the sequence of code points is encoded using a known character encoding

int32 vector - each position contains a single code point.


In [5]:
# Unicode string, represented as a vector of Unicode code points.
text_chars = tf.constant([ord(char) for char in u"hello"])
text_chars

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([104, 101, 108, 108, 111], dtype=int32)>

Converting between representations

TensorFlow provides operations to convert between these different representations:

**tf.strings.unicode_decode**: Converts an encoded string scalar to a vector of code points.

**tf.strings.unicode_encode**: Converts a vector of code points to an encoded string scalar.

**tf.strings.unicode_transcode**: Converts an encoded string scalar to a different encoding.

In [6]:
input =  [s.encode('utf8') for s in (u'G\xf6\xf6dight', u'\U0001f60a')]
tf.strings.unicode_decode(input,'UTF-8').to_list()

[[71, 246, 246, 100, 105, 103, 104, 116], [128522]]

In [7]:
input = tf.ragged.constant([[71, 246, 246, 100, 105, 103, 104, 116], [128522]])

print(tf.strings.unicode_encode(input, 'UTF-8'))

tf.Tensor([b'G\xc3\xb6\xc3\xb6dight' b'\xf0\x9f\x98\x8a'], shape=(2,), dtype=string)


In [8]:
tf.strings.unicode_transcode(["Hello",'TensorFlow',"2.x"], "UTF-8", "UTF-16-BE")

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'\x00H\x00e\x00l\x00l\x00o',
       b'\x00T\x00e\x00n\x00s\x00o\x00r\x00F\x00l\x00o\x00w',
       b'\x002\x00.\x00x'], dtype=object)>

<b>Unicode operations</b>

Character length 

The tf.strings.length operation has a parameter unit, which indicates how lengths should be computed. unit defaults to "BYTE", but it can be set to other values, such as "UTF8_CHAR" or "UTF16_CHAR", to determine the number of Unicode codepoints in each encoded string.



In [9]:
# Note that the final character takes up 4 bytes in UTF8.
thanks = u'Thanks 😊'.encode('UTF-8')
num_bytes = tf.strings.length(thanks).numpy()
num_chars = tf.strings.length(thanks, unit='UTF8_CHAR').numpy()
print('{} bytes; {} UTF-8 characters'.format(num_bytes, num_chars))

11 bytes; 8 UTF-8 characters


the tf.strings.substr operation accepts the "unit" parameter, and uses it to determine what kind of offsets the "pos" and "len" paremeters contain.

In [10]:
# default: unit='BYTE'. With len=1, we return a single byte.
tf.strings.substr(thanks, pos=7, len=1).numpy()

b'\xf0'

In [11]:
# Specifying unit='UTF8_CHAR', we return a single character, which in this case
# is 4 bytes.
print(tf.strings.substr(thanks, pos=7, len=1, unit='UTF8_CHAR').numpy())

b'\xf0\x9f\x98\x8a'


Split Unicode strings
The tf.strings.unicode_split operation splits unicode strings into substrings of individual characters

In [12]:
tf.strings.unicode_split(thanks, 'UTF-8').numpy()

array([b'T', b'h', b'a', b'n', b'k', b's', b' ', b'\xf0\x9f\x98\x8a'],
      dtype=object)

Unicode Scripts

Unicode code point belongs to a single collection of codepoints known as a script

TensorFlow provides the tf.strings.unicode_script operation to determine which script a given codepoint uses. The script codes are int32 values corresponding to International Components for Unicode (ICU) UScriptCode values.


In [13]:
uscript = tf.strings.unicode_script([33464, 1041]) 

print(uscript.numpy())  

[17  8]


The tf.strings.unicode_script operation can also be applied to multidimensional tf.Tensors or tf.RaggedTensors of codepoints:

In [14]:
print(tf.strings.unicode_script([1,31,39]))

tf.Tensor([0 0 0], shape=(3,), dtype=int32)
